main
- checkpoint and early stopping (stop when train loss decrease & val loss increase)
- save trained model
- compare model test performance
- check logger system of ignite (+ plot)

models
- efficientnet

readme
- arrayship-style image classification neural network pytorch coding convention
    - 2 or more ways > block parallel
    - 

## colab settings

In [1]:
import os

prj_name = 'cifar-10'
prj_path = '/content/drive/My Drive/colab/study/image_classification/'\
        + prj_name + '/'
os.chdir(prj_path + 'notebooks/')

## settings

In [2]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('..')

# main

In [3]:
from importlib import import_module

import torch
import torch.nn as nn
from torchvision import models

!pip install pytorch-ignite
from ignite.metrics import Accuracy, Loss

     |████████████████████████████████| 174kB 8.4MB/s 


#### VGG

In [ ]:
# model settings
model_mpath = 'src.models.vgg'
model_name = 'VGG'
model_cfg = {'cfg': [[64], [128], [256, 256], [512, 512], [512, 512]],
        'batch_norm': True}
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

#### InceptionV1

In [ ]:
# model settings
model_mpath = 'src.models.inception_v1'
model_name = 'InceptionV1'
model_cfg = [
        [{1: (0, 64), 3: (96, 128), 5: (16, 32), 'm': 32}, [False, 0, 0], None],
        [{1: (0, 128), 3: (128, 192), 5: (32, 96), 'm': 64}, [True, 3, 2], None],
        [{1: (0, 192), 3: (96, 208), 5: (16, 48), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 160), 3: (112, 224), 5: (24, 64), 'm': 64}, [False, 0, 0], 'aux'],
        [{1: (0, 128), 3: (128, 256), 5: (24, 64), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 112), 3: (144, 288), 5: (32, 64), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 256), 3: (160, 320), 5: (32, 128), 'm': 128}, [True, 2, 2], 'aux'],
        [{1: (0, 256), 3: (160, 320), 5: (32, 128), 'm': 128}, [False, 0, 0], None],
        [{1: (0, 384), 3: (192, 384), 5: (48, 128), 'm': 128}, [False, 0, 0], 'final']
        ]
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
class loss_cls(nn.Module):
    def __init__(self):
        super(loss_cls, self).__init__()
    def forward(self, inp, tar):
        loss = 0
        loss_fn = nn.CrossEntropyLoss()
        inps = inp.split(10, 1)
        for i, p in enumerate(inps):
            if i < len(inps) - 1:
                loss += 0.3 * loss_fn(p, tar)
            else:
                loss += 1.0 * loss_fn(p, tar)
        return loss
loss_fn = loss_cls()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

#### ResNet

In [ ]:
# model settings
model_mpath = 'src.models.resnet'
model_name = 'ResNet'
model_cfg = [
        [(3, 64), (3, 64)],
        [(3, 64), (3, 64)],
        [(3, 128), (3, 128)],
        [(3, 128), (3, 128)],
        [(3, 256), (3, 256)],
        [(3, 256), (3, 256)],
        [(3, 512), (3, 512)],
        [(3, 512), (3, 512)]
        ]
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000
# startblock batchnorm eps check

#### DenseNet

In [16]:
# model settings
model_mpath = 'src.models.densenet'
model_name = 'DenseNet'
gr = 4
cr = 0.5
model_cfg = {
        'start': 2 * gr,
        'dense': [
                [(4 * gr, gr)] * 4,
                [(4 * gr, gr)] * 8,
                [(4 * gr, gr)] * 12,
                #[(4 * gr, gr)] * 16
                ],
        'compress': cr
        }
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

## model construction

In [17]:
model_cls = getattr(
        import_module(model_mpath),
        model_name
        )
model = model_cls(model_cfg, init_weights=init_weights)
for b in model.named_children():
    print(b)

('B_000', DNStartBlock(
  (B_000): Conv2d(3, 8, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (B_001): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (B_002): ReLU(inplace=True)
  (B_003): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('B_001', DNDenseBlock(
  (B_000): DNCompositeBlock(
    (B_000): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_001): ReLU(inplace=True)
    (B_002): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (B_003): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_004): ReLU(inplace=True)
    (B_005): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (B_001): DNCompositeBlock(
    (B_000): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_001): ReLU(inplace=True)
    (B_002): Conv2d(12, 16, kernel_size=(1, 1), stride=(

## load data

In [6]:
from src.data.make_dataset import Cifar10BatchDataset

dpath = '../data/raw/'

train_folds = []
for i in range(1, 6):
    train_folds.append(torch.load(dpath + 'data_batch_' + str(i) + '.pt'))
test_set = torch.load(dpath + 'test_batch.pt')

In [7]:
from collections import Counter

for train_fold in train_folds:
    print(sorted(Counter(s['label'] for s in train_fold).items()))
print(sorted(Counter(s['label'] for s in test_set).items()))

[(0, 1005), (1, 974), (2, 1032), (3, 1016), (4, 999), (5, 937), (6, 1030), (7, 1001), (8, 1025), (9, 981)]
[(0, 984), (1, 1007), (2, 1010), (3, 995), (4, 1010), (5, 988), (6, 1008), (7, 1026), (8, 987), (9, 985)]
[(0, 994), (1, 1042), (2, 965), (3, 997), (4, 990), (5, 1029), (6, 978), (7, 1015), (8, 961), (9, 1029)]
[(0, 1003), (1, 963), (2, 1041), (3, 976), (4, 1004), (5, 1021), (6, 1004), (7, 981), (8, 1024), (9, 983)]
[(0, 1014), (1, 1014), (2, 952), (3, 1016), (4, 997), (5, 1025), (6, 980), (7, 977), (8, 1003), (9, 1022)]
[(0, 1000), (1, 1000), (2, 1000), (3, 1000), (4, 1000), (5, 1000), (6, 1000), (7, 1000), (8, 1000), (9, 1000)]


In [8]:
from torch.utils.data import ConcatDataset, DataLoader

train_set = ConcatDataset([f for j, f in enumerate(train_folds) if j != 4])
val_set = train_folds[4]

train_loader = DataLoader(train_set, batch_size=batch_size,
        shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size,
        shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size,
        shuffle=True, num_workers=2)

del train_folds

## train

In [ ]:
from src.models.train_model import train_net

opt = opt_(model.parameters(), lr)

trainer = train_net(model, opt, loss_fn, val_metrics,
        train_loader, val_loader, device)
trainer.run(train_loader, max_epochs=max_epochs)

Epoch 1
Train - acc: 0.11 loss: 2.33 
Val   - acc: 0.11 loss: 2.33 
Epoch 2
Train - acc: 0.12 loss: 2.32 
Val   - acc: 0.12 loss: 2.32 
Epoch 3
Train - acc: 0.13 loss: 2.31 
Val   - acc: 0.13 loss: 2.31 
Epoch 4
Train - acc: 0.14 loss: 2.30 
Val   - acc: 0.14 loss: 2.29 
Epoch 5
Train - acc: 0.14 loss: 2.28 
Val   - acc: 0.15 loss: 2.28 
Epoch 6
Train - acc: 0.15 loss: 2.27 
Val   - acc: 0.15 loss: 2.27 
Epoch 7
Train - acc: 0.15 loss: 2.26 
Val   - acc: 0.16 loss: 2.25 
Epoch 8
Train - acc: 0.16 loss: 2.24 
Val   - acc: 0.17 loss: 2.24 
Epoch 9
Train - acc: 0.17 loss: 2.23 
Val   - acc: 0.17 loss: 2.23 
Epoch 10
Train - acc: 0.17 loss: 2.22 
Val   - acc: 0.17 loss: 2.22 
Epoch 11
Train - acc: 0.18 loss: 2.21 
Val   - acc: 0.18 loss: 2.21 
Epoch 12
Train - acc: 0.18 loss: 2.20 
Val   - acc: 0.19 loss: 2.20 
Epoch 13
Train - acc: 0.19 loss: 2.18 
Val   - acc: 0.19 loss: 2.19 
Epoch 14
Train - acc: 0.19 loss: 2.17 
Val   - acc: 0.20 loss: 2.17 
Epoch 15
Train - acc: 0.20 loss: 2.16 
Val 

## Test